# Reconocimiento de comentarios negativos

En esta libreta vamos a utilizar una red neuronal profunda para tratar de detectar los comentarios negativos.

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Para el tratamiento del lenguaje
from nltk.corpus import stopwords
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import sklearn.metrics as metricas


## Paso 1: Recuperar y tratar los datos para usarse

Los datos se encuentran en un directorio privado, los cuales ya se 
encuentran separados de un conjunto de prueba. 

Los datos se encuentran como una tabla tipo csv de la siguiente manera:

Columnas  |    Nombre       |     Descripción
----------|-----------------|----------------------------------------------------------
1         |   id            |    Identificador del mensaje
2         |   create_time   |    Fecha de creación del mensaje
3         |   id_from       |    Identificador del usuario (emisor del mensaje)
4         |   comment       |    El mensaje en si mismo
5         |   like_count    |    Número de likes que recibió el mensaje
6         |   flag          |    Polaridad (0 -> Negativo, 1 -> Positivo, 2 -> Neutro)
7         |   ready         |    Revisado por un operador (1 -> No, 2 -> Si)

Una vez recuperados los datos, se realiza el siguiente pretratamiento:

1. La columna `flag` será el valor de salida y la columna `comment` 
   como variables de entrada, las cuales son string. 
   
2. La columna `comment`la vamos a tratar, ya que cada usuario maneja 
   codificaciones diferentes y los datos están hechos un chile con queso.

3. La columna `flag` se le asigna valor de 0 a los valores 1 y 2 
   (comentarios no negativos) y valor de 1 a los que tienen valor de 0
   (comentarios negativos). Convertimos el problema a un problema de
   clasificación binaria.
   
4. Los datos se separan, 80% para entrenamiento y 20% para validación, 
   procurando que en ambos conjuntos haya la misma proporción de 
   elementos de ambas clases.


In [27]:
# Cargamos los datos
df = pd.read_csv("privado/data_train.csv")

X = df["comment"]
X = X.apply(lambda x: str(x).decode('unicode_escape').encode('utf-8', 'ignore').strip())

y = np.array(df['flag'])

# Problema binario 1 = Comentario negativo
y = np.where(y > 0, 0, 1)

# Separamos 20% para validación
indices = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=0)
for tr_i, va_i in indices:
    X_train, X_valid = X.loc[tr_i], X.loc[va_i]
    y_train, y_valid = y[tr_i], y[va_i]
    break


## Paso 2: Genera la serie de operaciones a realizar.

En esta paso vamos a decidir cuales métodos vamos a utilizar para tratar la información,
y cual método vamos a utilizar para la clasificación. Los métodos que se seleccionen serán
incluídos en un `pipeline` que nos ayuda a serializar las operaciones a realizar.

Entre los métodos de vectorización de la información que podemos utilizar se encuentran:

1. Bolsa de palabras (o bigramas): `CountVectorizer`
2. Tf–idf term weighting: `TfidfVectorizer`

Entre los métodos de reducción de la dimensionalidad se encuentran:

1. Análisis en componentes principales para datos dispersos: `SparsePCA`
2. Latent semantic analysis (LSA): `TruncatedSVD`

Estos pueden ser utilizados en forma combinada. Los métodos de clasificación pueden ser:

1. Máquina de Vectores de Soporte Lineal: `LinearSVC`
2. Máquina de Vectores de Soporte: `SVC`
3. Naive Bayes: `MultinomialNB`
4. Bosques aleatorios: `RandomForestClassifier`
5. AdaBoost: `AdaBoostClassifier`
6. Árbol de desición: `DecisionTreeClassifier`

Por último, es necesario decidir si hay parámetros que pueden ser modificados, y en que términos pueden ser modificados, esto se agrega en el diccionario `parameters`.

In [59]:
espanol_stopwords = stopwords.words('spanish')
vectorizer = TfidfVectorizer(
    analyzer='word',
    lowercase=True,
    stop_words=espanol_stopwords)

reductor = TruncatedSVD(n_components=500)

In [79]:
cls = tf.contrib.learn.TensorFlowDNNClassifier(
    hidden_units=[10, 10, 10],
    n_classes=2,
    optimizer="Adam",
    class_weight=[.9, 0.1],
    steps=2000)

## Paso 3. Ejecutar preprocesamiento



In [82]:
X_vec = vectorizer.fit_transform(X_train)
X_red = reductor.fit_transform(X_vec)

## Paso: 4 Ejecutar aprendizaje

In [83]:
cls.fit(X=X_red, y=y_train)

Step #99, avg. train loss: 0.26188
Step #199, avg. train loss: 0.21683
Step #299, avg. train loss: 0.20210
Step #399, avg. train loss: 0.21823
Step #499, avg. train loss: 0.22198
Step #599, avg. train loss: 0.19774
Step #699, avg. train loss: 0.20516
Step #799, avg. train loss: 0.20313
Step #899, avg. train loss: 0.19660
Step #999, avg. train loss: 0.20212
Step #1099, avg. train loss: 0.21078
Step #1199, avg. train loss: 0.18020
Step #1299, avg. train loss: 0.19251
Step #1399, avg. train loss: 0.19197
Step #1499, avg. train loss: 0.20233
Step #1599, avg. train loss: 0.20070
Step #1699, avg. train loss: 0.20067
Step #1800, epoch #1, avg. train loss: 0.20026
Step #1900, epoch #1, avg. train loss: 0.21340
Step #2000, epoch #1, avg. train loss: 0.20582


TensorFlowDNNClassifier(batch_size=32, class_weight=[0.9, 0.1],
            clip_gradients=5.0, config=None, continue_training=False,
            dropout=None, hidden_units=[10, 10, 10], learning_rate=0.1,
            n_classes=2, optimizer='Adam', steps=2000, verbose=1)

## Paso 4: Analisis de los resultados

Por último analizamos que tan bien se desempeña en los datos de validación.


In [84]:
# Muestra los resultados con datos de validacion
X_vec = vectorizer.transform(X_valid)
X_red = reductor.transform(X_vec)
y_est = cls.predict(X_red)
print metricas.confusion_matrix(y_valid, y_est)


[[12583   342]
 [  775   452]]
